In [1]:
%cd ~/work/deeplearning/

/home/asaeed9/work/deeplearning


In [2]:
from theano.sandbox import cuda
cuda.use('gpu1')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

ERROR (theano.sandbox.cuda): ERROR: Not using GPU. Initialisation of device 1 failed:
Bad device number 1. Only 1 devices available.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 980 Ti (CNMeM is disabled, cuDNN 5105)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to

## Headers

In [3]:
%matplotlib inline
from __future__ import print_function, division
#path="../data/2cat/sample"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink
from keras.preprocessing import image, sequence
import os, sys
from shutil import copyfile

####
from keras.layers.convolutional import *
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam

Using Theano backend.


In [4]:
batch_size=16

In [5]:
path="/home/asaeed9/work/data/2cat/sample/"
results_path = "/home/asaeed9/work/data/2cat/sample/results"
test_path = path + '/test/' #We use all the test data

In [6]:
%cd ../data/2cat/train

/home/asaeed9/work/data/2cat/train


## Data Preparation

In [20]:
#copy training images
def copy_samples(n):
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(n): copyfile(shuf[i], '../sample/train/' + shuf[i]) 
    %mv ../sample/train/cat*.jpg ../sample/train/cats/
    %mv ../sample/train/dog*.jpg ../sample/train/dogs/
    
    %cd ../valid

    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(500): copyfile(shuf[i], '../sample/valid/' + shuf[i]) 
    %mv ../sample/valid/cat*.jpg ../sample/valid/cats/
    %mv ../sample/valid/dog*.jpg ../sample/valid/dogs/
    %cd /home/asaeed9/work/data/2cat/train

In [21]:
def clean_up():
    %rm $path/train/cats/*
    %rm $path/train/dogs/*
    %rm $path/valid/cats/*
    %rm $path/valid/dogs/*

In [22]:
clean_up()

In [23]:
%cd /home/asaeed9/work/data/2cat/train
#for n in range(1000, 5001, 1000):
n = 1500
print('sample size: {}'.format(n))
copy_samples(n)

/home/asaeed9/work/data/2cat/train
sample size: 1500
/home/asaeed9/work/data/2cat/valid
/home/asaeed9/work/data/2cat/train


## Training 

In [33]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,256,256)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
#             Convolution2D(64,3,3, activation='relu'),
#             BatchNormalization(axis=1),
            #MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Dropout(0.2),
            Flatten(),
            Dense(1024, activation='relu'),
            BatchNormalization(),
            Dropout(0.2),
            Dense(2, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
#     model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
#                      nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=50, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [34]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

tr_batches = gen_t.flow_from_directory(path + 'train', batch_size=batch_size)
val_batches = gen_t.flow_from_directory(path + 'valid', class_mode='categorical', shuffle=True, batch_size=batch_size * 2)

Found 1500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [38]:
model = conv1(tr_batches)
model.save_weights(results_path+ '/' +'ft_15000.e50')

Epoch 1/50
1500/1500 [==============================] - 28s - loss: 1.2398 - acc: 0.5813 - val_loss: 2.1502 - val_acc: 0.4900
Epoch 2/50
1500/1500 [==============================] - 27s - loss: 0.7863 - acc: 0.6020 - val_loss: 1.6568 - val_acc: 0.4980
Epoch 3/50
1500/1500 [==============================] - 30s - loss: 0.7426 - acc: 0.6260 - val_loss: 1.7099 - val_acc: 0.4820
Epoch 4/50
1500/1500 [==============================] - 25s - loss: 0.6616 - acc: 0.6493 - val_loss: 0.9976 - val_acc: 0.5820
Epoch 5/50
1500/1500 [==============================] - 26s - loss: 0.6466 - acc: 0.6640 - val_loss: 0.8389 - val_acc: 0.6700
Epoch 6/50
1500/1500 [==============================] - 26s - loss: 0.6149 - acc: 0.6840 - val_loss: 1.3903 - val_acc: 0.5700
Epoch 7/50
1500/1500 [==============================] - 25s - loss: 0.5838 - acc: 0.7060 - val_loss: 0.7206 - val_acc: 0.7020
Epoch 8/50
1500/1500 [==============================] - 25s - loss: 0.5738 - acc: 0.7107 - val_loss: 0.7307 - val_acc:

## Prediction

In [16]:
gen_test = image.ImageDataGenerator()
test_batches = gen_test.flow_from_directory(test_path, batch_size=batch_size)

Found 3000 images belonging to 2 classes.


In [34]:
model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,256,256)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
#             Convolution2D(64,3,3, activation='relu'),
#             BatchNormalization(axis=1),
            #MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(1024, activation='relu'),
            BatchNormalization(),
            Dense(2, activation='softmax')
        ])
    
model.load_weights(results_path+'/ft_15000.e50')    
preds = model.predict(np.array(test_batches))

TypeError: 'axis' is an invalid keyword argument for this function

In [30]:
print(preds[:10])
print(test_batches.filenames[:10])
print(test_batches.classes[:10])
#is dog
# print(preds[:10])
# y_pred = np.argmax(preds, axis=1)
#y_pred[:10]

[[ 0.9899  0.0101]
 [ 0.0016  0.9984]
 [ 0.0097  0.9903]
 [ 0.1045  0.8955]
 [ 0.003   0.997 ]
 [ 0.985   0.015 ]
 [ 0.6525  0.3475]
 [ 0.3544  0.6456]
 [ 0.9281  0.0719]
 [ 0.9812  0.0188]]
['cats/cat.10923.jpg', 'cats/cat.12414.jpg', 'cats/cat.6013.jpg', 'cats/cat.2881.jpg', 'cats/cat.2445.jpg', 'cats/cat.11129.jpg', 'cats/cat.359.jpg', 'cats/cat.9922.jpg', 'cats/cat.12013.jpg', 'cats/cat.701.jpg']
[0 0 0 0 0 0 0 0 0 0]


In [ ]:
model.predict_classes()

In [15]:
filenames = test_batches.filenames
np.set_printoptions(suppress=True, precision=4)
print(filenames[:10])
print(preds[:10])
#our_labels = np.round(preds[:, 0])
#our_labels[:10]
#our_preds = preds[:, 1]

['unknown/cat.10923.jpg', 'unknown/dog.571.jpg', 'unknown/cat.12414.jpg', 'unknown/cat.6013.jpg', 'unknown/cat.2881.jpg', 'unknown/dog.3678.jpg', 'unknown/cat.2445.jpg', 'unknown/cat.11129.jpg', 'unknown/cat.359.jpg', 'unknown/cat.9922.jpg']
[[ 0.0184  0.9816]
 [ 0.002   0.998 ]
 [ 0.8797  0.1203]
 [ 0.9809  0.0191]
 [ 0.0001  0.9999]
 [ 0.0099  0.9901]
 [ 0.9102  0.0898]
 [ 0.3978  0.6022]
 [ 0.6796  0.3204]
 [ 0.0134  0.9866]]


In [48]:
#print(preds[:10])
filenames = test_batches.filenames
np.set_printoptions(suppress=True, precision=4)
filenames[:10]

['unknown/cat.10923.jpg',
 'unknown/dog.571.jpg',
 'unknown/cat.12414.jpg',
 'unknown/cat.6013.jpg',
 'unknown/cat.2881.jpg',
 'unknown/dog.3678.jpg',
 'unknown/cat.2445.jpg',
 'unknown/cat.11129.jpg',
 'unknown/cat.359.jpg',
 'unknown/cat.9922.jpg']

In [44]:
expected_labels = get_test_labels(test_batches.filenames)

NameError: name 'get_test_labels' is not defined